<a href="https://colab.research.google.com/github/Nnamaka/object_detection_with_yolo/blob/main/Yolo_Hand_Gestures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Hand Gesture Reccognition with YOLOv5


This notebook is a run-through of the major steps to perform a complete training pipeline for a custom object detection YOLO model using colab. 

I wrote a blog tutorial describing details of the procedure for training a Yolov5 model on your custom dataset. 
Check it out here--------

The basic steps are:

* Install Yolo and its Requirments
* Annotate dataset and serve if from Roboflow
* Train Yolo for custom detections
* Infer the new trainded model weights

##Step-1: Install Yolo from its Repository

In [ ]:
import torch
import os
from IPython.display import Image, clear_output

GPU?

In [ ]:
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

clone her

In [ ]:
# clone yolo5
!git clone https://github.com/ultralytics/yolov5  


Install requirments

In [ ]:
%cd yolov5
%pip install -qr requirements.txt
%pip install -q roboflow

##Step-2: Fetch Annotated Dataset from Roboflow

For my dataset of hand gestures, I took a few pictures with my mobile phone, sent them to my PC and used the <a href="https://github.com/tzutalin/labelImg">labelImg</a> tool to label them. labelImg has the function to label Image datasets to the PascalVOC format and also the Yolo format.



I an option to also export my raw image dataset to Roboflow and label them there. But I chose to do it in LabelImg. 

After Labeling my dataset, I exported it to <a href="https://roboflow.com/">Roboflow</a>. A Great platform that addresses major pre-processing requirments for image dataset.

copy the code from Roboflow and paste it in the cell below

In [ ]:
# paste your api key in the variable below
api_key = 'paste your api key here'

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key=api_key)
project = rf.workspace("gestures-kejav").project("hand_gestures-iwirp")
dataset = project.version(1).download("yolov5")

In [ ]:
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

##Step-3: Train

In [ ]:
!python train.py --img 416 --batch 8 --epochs 150 --data {dataset.location}/data.yaml --weights yolov5s.pt --cache

##Step-4: Evaluate trained model

While training goes on, Performance metrics are logged(saved in 'runs' folder)
You can choose to run the cell below first before begining/running the training cell. This will enable you to monitor the model performance while training.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

##Step-5: Inference with trained model

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.1 --source {dataset.location}/test/images

Let us view the inference result

In [ ]:
import glob
from IPython.display import Image, display

#change your the extension variable to your image type
extension = '.jpg'
for imageName in glob.glob('/content/yolov5/runs/detect/exp/*' + extension):
    display(Image(filename=imageName))
    print("\n")

##Step-6: Final step. Download your trained model

Your model is served!

In [ ]:
from google.colab import files
files.download('./runs/train/exp/weights/best.pt')

##Step-7: Conclusion

What?
YOLO is amazing!

I noticed the model didn't do well in the 'ThumbsUp' hand-gesture prediction. The differents ways to solve that are:

* Increase the images for 'ThumbsUp'
* Run the training for Longer epochs
* Augument the 'ThumbsUp' images eg different light exposure, contrast, bluring , cropping , rotaion etc. This will give the model enough variated data to look at and learn from.